In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

warnings.filterwarnings('ignore')

In [47]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Load datasets
ratings = pd.read_csv("drive/MyDrive/Dataset2/ratings.csv")
movies = pd.read_csv("drive/MyDrive/Dataset2/movies.csv")

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [86]:
movies.head(48)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
# Preprocess ratings data
ratings = ratings[['userId', 'movieId', 'rating']]

In [7]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [8]:
# Filter users with more than 200 ratings
x = ratings['userId'].value_counts() > 200
y = x[x].index
ratings = ratings[ratings['userId'].isin(y)]

In [9]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
# Merge movies and ratings data
movies_ratings = pd.merge(ratings, movies, on="movieId")
movies_ratings.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,18,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,19,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,21,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,45,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [11]:
# Count number of ratings for each movie
number_rating = movies_ratings.groupby('title')['rating'].count().reset_index()
number_rating.head()

,title,rating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),1
3,'Tis the Season for Love (2015),1
4,"'burbs, The (1989)",12


In [12]:
number_rating.rename(columns={'rating': 'No.Of Rating'}, inplace=True)
number_rating.head()

,title,No.Of Rating
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),1
3,'Tis the Season for Love (2015),1
4,"'burbs, The (1989)",12


In [40]:
# Merge to get final ratings data
final_rating = movies_ratings.merge(number_rating, on='title')
final_rating.head()

,userId,movieId,rating,title,genres,No.Of Rating
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92
1,18,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92
2,19,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92
3,21,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92
4,45,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,92


In [17]:
final_rating = final_rating[final_rating['No.Of Rating'] >= 50]
final_rating.head()

,userId,movieId,rating,title,genres,No.Of Rating
718,1,260,5.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,111
719,4,260,5.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,111
720,18,260,4.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,111
721,19,260,4.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,111
722,21,260,4.5,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,111


In [68]:
# Create the user-movie interaction matrix
user_movie_matrix = final_rating.pivot_table(columns='movieId', index='userId', values='rating')
user_movie_matrix.head()

movieId,1,2,6,10,32,34,47,50,104,110,111,150,153,165,223,231,260,293,296,316,318,344,356,357,364,367,377,380,457,480,500,527,541,551,586,587,588,589,590,592,593,595,597,608,648,733,736,750,778,780,858,912,919,924,1028,1036,1073,1079,1080,1089,1097,1101,1136,1193,1196,1197,1198,1200,1206,1208,1210,1213,1214,1219,1220,1221,1222,1240,1246,1258,1259,1265,1270,1291,1307,1387,1391,1393,1485,1517,1527,1544,1573,1580,1584,1610,1617,1653,1676,1682,1704,1721,1732,1784,1917,1923,1961,1968,2000,2011,2012,2028,2081,2115,2174,2291,2329,2355,2396,2502,2571,2617,2628,2683,2700,2706,2716,2762,2791,2797,2858,2916,2918,2959,2985,2987,2997,3052,3114,3147,3175,3481,3578,3623,3751,3793,3897,3948,3949,3977,3994,3996,4011,4022,4027,4226,4306,4878,4886,4896,4963,4973,4979,4993,4995,5218,5349,5378,5418,5445,5481,5502,5816,5952,5989,6333,6365,6373,6377,6539,6874,6934,7153,7361,7438,8360,8368,8636,8874,8961,32587,33493,33794,44191,45722,48516,48780,51662,58559,59315,60069,68954,69122,72998,79132
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,NaN,5.0,5.0,NaN,4.0,NaN,NaN,NaN,NaN,3.0,5.0,5.0,NaN,3.0,3.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,5.0,4.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,NaN,NaN,5.0,3.0,4.0,3.0,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,5.0,NaN,5.0,5.0,5.0,NaN,5.0,NaN,5.0,5.0,5.0,NaN,5.0,4.0,5.0,5.0,4.0,2.0,5.0,NaN,5.0,5.0,NaN,3.0,NaN,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,3.0,NaN,NaN,5.0,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,4.0,NaN,5.0,4.0,5.0,5.0,NaN,NaN,5.0,5.0,2.0,4.0,NaN,5.0,NaN,5.0,NaN,NaN,4.0,5.0,4.0,NaN,5.0,4.0,5.0,4.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,3.0,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,4.0,2.0,5.0,NaN,NaN,NaN,5.0,NaN,5.0,5.0,3.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,4.0,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,4.0,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,4.0,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,2.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,4.0,4.0,3.0,4.0,4.0,4.0,1.0,4.0,5.0,NaN,4.0,3.0,3.0,NaN,3.0,NaN,3.0,2.0,5.0,5.0,3.0,5.0,3.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,3.0,NaN,NaN,NaN,5.0,5.0,3.0,5.0,3.0,4.0,5.0,4.0,3.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,3.5,3.0,4.0,NaN,4.0,2.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,4.0,4.0,2.5,4.0,4.5,4.0,NaN,5.0,2.5,4.5,NaN,3.0,2.5,2.5,2.5,3.5,3.5,3.5,4.5,4.0,NaN,3.5,NaN,3.5,4.5,4.0,NaN,4.5,3.0,NaN,4.5,3.0,NaN,NaN,NaN,4.5,4.0,4.0,NaN,4.0,4.0,NaN,4.0,NaN,NaN,4.5,4.0,NaN,NaN,4.5,4.5,4.0,NaN,4.0,NaN,4.5,3.5,4.5,4.5,4.0,4.5,NaN,5.0,4.5,4.0,4.0,NaN,NaN,NaN,4.0,4.0,

In [69]:
user_movie_matrix.fillna(0, inplace=True)
user_movie_matrix.head()

movieId,1,2,6,10,32,34,47,50,104,110,111,150,153,165,223,231,260,293,296,316,318,344,356,357,364,367,377,380,457,480,500,527,541,551,586,587,588,589,590,592,593,595,597,608,648,733,736,750,778,780,858,912,919,924,1028,1036,1073,1079,1080,1089,1097,1101,1136,1193,1196,1197,1198,1200,1206,1208,1210,1213,1214,1219,1220,1221,1222,1240,1246,1258,1259,1265,1270,1291,1307,1387,1391,1393,1485,1517,1527,1544,1573,1580,1584,1610,1617,1653,1676,1682,1704,1721,1732,1784,1917,1923,1961,1968,2000,2011,2012,2028,2081,2115,2174,2291,2329,2355,2396,2502,2571,2617,2628,2683,2700,2706,2716,2762,2791,2797,2858,2916,2918,2959,2985,2987,2997,3052,3114,3147,3175,3481,3578,3623,3751,3793,3897,3948,3949,3977,3994,3996,4011,4022,4027,4226,4306,4878,4886,4896,4963,4973,4979,4993,4995,5218,5349,5378,5418,5445,5481,5502,5816,5952,5989,6333,6365,6373,6377,6539,6874,6934,7153,7361,7438,8360,8368,8636,8874,8961,32587,33493,33794,44191,45722,48516,48780,51662,58559,59315,60069,68954,69122,72998,79132
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,5.0,5.0,0.0,3.0,3.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,5.0,4.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0,0.0,5.0,3.0,4.0,3.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0,0.0,5.0,5.0,5.0,0.0,5.0,4.0,5.0,5.0,4.0,2.0,5.0,0.0,5.0,5.0,0.0,3.0,0.0,4.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,3.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,4.0,0.0,5.0,4.0,5.0,5.0,0.0,0.0,5.0,5.0,2.0,4.0,0.0,5.0,0.0,5.0,0.0,0.0,4.0,5.0,4.0,0.0,5.0,4.0,5.0,4.0,5.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,5.0,3.0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,4.0,2.0,5.0,0.0,0.0,0.0,5.0,0.0,5.0,5.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,4.0,0.0,0.0,0.0,4.0,5.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,4.0,3.0,4.0,4.0,4.0,1.0,4.0,5.0,0.0,4.0,3.0,3.0,0.0,3.0,0.0,3.0,2.0,5.0,5.0,3.0,5.0,3.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,3.0,0.0,0.0,0.0,5.0,5.0,3.0,5.0,3.0,4.0,5.0,4.0,3.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,3.5,3.0,4.0,0.0,4.0,2.5,4.5,5.0,2.5,4.5,4.0,0.0,0.0,4.0,4.0,2.5,4.0,4.5,4.0,0.0,5.0,2.5,4.5,0.0,3.0,2.5,2.5,2.5,3.5,3.5,3.5,4.5,4.0,0.0,3.5,0.0,3.5,4.5,4.0,0.0,4.5,3.0,0.0,4.5,3.0,0.0,0.0,0.0,4.5,4.0,4.0,0.0,4.0,4.0,0.0,4.0,0.0,0.0,4.5,4.0,0.0,0.0,4.5,4.5,4.0,0.0,4.0,0.0,4.5,3.5,4.5,4.5,4.0,4.5,0.0,5.0,4.5,4.0,4.0,0.0,0.0,0.0,4.0,4.0,

In [72]:
# Convert to sparse matrix
user_movie_sparse = csr_matrix(user_movie_matrix)

In [73]:
# Fit the collaborative filtering model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(user_movie_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [77]:
# Define the collaborative filtering function
def collaborative_filtering(user_id):
    if user_id not in user_movie_matrix.index:
        print(f"User ID {user_id} not found in user_movie_matrix.")
        return

    # Get the distances and indices of the nearest neighbors
    distances, suggestions = model.kneighbors(user_movie_matrix.loc[user_id].values.reshape(1, -1), n_neighbors=6)

    # Get the movies this user has interacted with
    user_interactions = user_movie_matrix.loc[user_id].to_numpy().nonzero()[0]
    recommended_movies = []

    # Loop through the suggestions (similar users)
    for i in range(1, len(suggestions[0])):  # Skip the first user (itself)
        similar_user_id = suggestions[0][i]
        similar_user_interactions = user_movie_matrix.iloc[similar_user_id].to_numpy().nonzero()[0]

        # Find recommendations (movies that similar users have rated, but this user has not)
        recommendations = np.setdiff1d(similar_user_interactions, user_interactions)
        recommended_movies.extend(recommendations)

    # Remove duplicates and limit to top 10 recommendations
    recommended_movies = list(set(recommended_movies))[:10]

    if not recommended_movies:
        print(f"No recommendations available for User {user_id}.")
        return

    print(f'The Suggestions For User {user_id} Are:')
    for movie_id in recommended_movies:
        if movie_id in movies['movieId'].values:
            movie_title = movies[movies['movieId'] == movie_id]['title'].values[0]
            print(movie_title)
        else:
            print(f"Movie ID {movie_id} not found in movies dataset.")


In [84]:
# Example usage
user_id = int(input('Enter the user id : '))
collaborative_filtering(user_id)

Enter the user id : 1
The Suggestions For User 1 Are:
Toy Story (1995)
Grumpier Old Men (1995)
Waiting to Exhale (1995)
Father of the Bride Part II (1995)
Tom and Huck (1995)
GoldenEye (1995)
American President, The (1995)
Dracula: Dead and Loving It (1995)
Balto (1995)
Sense and Sensibility (1995)
